In [79]:
import numpy as np
import pandas as pd
from datetime import datetime

# settings
datadir = "../../SmartCareData/"

In [224]:
# Extract data and format datatypes
# REFACTOR: if I don't format SmartCareID with convert_dtypes(), then it is an object rather than a string. The solution can be to manually enforce types instead of using convert_dtypes()
id_map = pd.read_excel(datadir + "patientidnew.xlsx", dtype={'SmartCareID': str}).drop("Study_ID", axis=1).convert_dtypes()
measurements = pd.read_csv(datadir + "mydata.csv").convert_dtypes()
measurements['Date/Time recorded'] = pd.to_datetime(measurements['Date/Time recorded']).dt.date

# Map smartcare ids
measurements = measurements.merge(id_map, left_on='User ID', right_on='Patient_ID', copy=True)
measurements

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_46758/1619914002.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  measurements = pd.read_csv(datadir + "mydata.csv").convert_dtypes()


,User ID,UserName,Recording Type,Date/Time recorded,FEV 1,FEV 10,Predicted FEV,FEV 1 %,Weight in Kg,O2 Saturation,Pulse (BPM),Calories,Rating,Temp (deg C),Sputum sample taken?,Activity - Steps,Activity - Points,Patient_ID,SmartCareID
0,VtussNQdJTdwKO4LuYroh4,Papworth004,ActivityRecording,2015-08-05,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,258,65,VtussNQdJTdwKO4LuYroh4,79
1,VtussNQdJTdwKO4LuYroh4,Papworth004,ActivityRecording,2015-08-06,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,610,163,VtussNQdJTdwKO4LuYroh4,79
2,VtussNQdJTdwKO4LuYroh4,Papworth004,ActivityRecording,2015-08-07,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,VtussNQdJTdwKO4LuYroh4,79
3,VtussNQdJTdwKO4LuYroh4,Papworth004,ActivityRecording,2015-08-08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,VtussNQdJTdwKO4LuYroh4,79
4,VtussNQdJTdwKO4LuYroh4,Papworth004,ActivityRecording,2015-08-09,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4,1,VtussNQdJTdwKO4LuYroh4,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123142,c8uT2-nIew-wE2huQZXcy4,010wessex,PulseRateRecording,2016-09-30,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,76,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,c8uT2-nIew-wE2huQZXcy4,199
123143,c8uT2-nIew-wE2huQZXcy4,010wessex,SleepActivityRecording,2016-09-30,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,75,<NA>,<NA>,<NA>,<NA>,c8uT2-nIew-wE2huQZXcy4,199
123144,c8uT2-nIew-wE2huQZXcy4,010wessex,SputumSampleRecording,2016-09-30,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,True,<NA>,<NA>,c8uT2-nIew-wE2huQZXcy4,199
123145,c8uT2-nIew-wE2huQZXcy4,010wessex,WeightRecording,2016-09-30,<NA>,<NA>,<NA>,<NA>,45.8125,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,c8uT2-nIew-wE2huQZXcy4,199


In [268]:
# Get table with FEV1 and O2 values per day per user

# Extract data for one column
def extract_measure(label):
    # Could also filter by Recording Type
    data = measurements[measurements[ label ].notnull()][['SmartCareID', 'Date/Time recorded', label]]
    return data

O2 = extract_measure("O2 Saturation")
FEV1 = extract_measure("FEV 1")
length_outer_join = O2.merge(FEV1, on=['SmartCareID', 'Date/Time recorded'], how='outer').shape[0]
length_left_join  = O2.merge(FEV1, on=['SmartCareID', 'Date/Time recorded'], how='left').shape[0]
O2_FEV1 = O2.merge(FEV1, on=['SmartCareID', 'Date/Time recorded'], how='inner')
print("{} O2-FEV1 same day measurements. Lost {:.0%} of rows with inner merge".format( length_outer_join, (length_outer_join - O2_FEV1.shape[0])/O2_FEV1.shape[0]))

14791 O2-FEV1 same day measurements. Lost 46% of rows with inner merge


In [269]:
O2_FEV1

,SmartCareID,Date/Time recorded,O2 Saturation,FEV 1
0,79,2015-09-10,95,1.53
1,79,2015-09-11,97,1.68
2,79,2015-09-12,97,1.48
3,79,2015-08-13,95,1.63
4,79,2015-11-07,94,1.52
...,...,...,...,...
10100,72,2016-06-23,94,1.14
10101,72,2016-07-23,90,1.2
10102,72,2016-06-21,95,1.02
10103,72,2016-07-08,96,1.23
